In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [132]:
def viscosity(Tlava, Tcurrent, phi_crystal, phi_crystal_max, eta0):
    a = 0.04  #in K-1
    eta = eta0 * (1-(phi_crystal / phi_crystal_max)**-2.5) * np.exp(a * (Tlava - Tcurrent))

def yield_strength(Tlava, Tcurrent, phi_crystal):
    a = 1e-2 ## in PA
    b = 0.08 ## in K-1
    ys = (6500*phi_crystal**2.85) + a*np.exp(b*(Tlava-Tcurrent)-1)
    return ys

def basal_shear_stress(rho, g, h, theta_deg):
    return rho * g * h * np.sin(np.rad2deg(theta_deg))

def velocity_Newtonian(rho, h, eta, theta_deg, C = 3):
    ## Jeffreys equation
    ## C = 3 for a filled channel
    ## C=8 for a wide flow with a moving top
    ## C = 12 for a sheet flow
    vel = rho*g*h**2*np.sin(np.deg2rad(theta_deg)) / C / eta
    return vel

def velocity_Bingham(rho, g, h, eta, theta_deg, yield_strength):
    c1 = 4/3
    c2 = 1/3
    c3 = 4
    basal_stress = basal_shear_stress(rho, g, h, theta_deg)
    vel = rho*g*h**2*np.sin(theta_deg) / (3*eta) * (1 - c1*(yield_strength/basal_stress) + c2*(yield_strength/basal_stress)**c3)
    return vel

def levee_width(rho, g, eta, theta_deg, yield_strength, effusion_rate):
    wb = yield_strength / (2 * g * rho * np.deg2rad(theta_deg)**2)
    print(2 * g * rho * np.deg2rad(theta_deg)**2)
    return wb
    
def levee_thickness(rho, g, eta, theta_deg, yield_strength, effusion_rate):
    h = yield_strength / g / rho/ np.deg2rad(theta_deg)
    return h

def channel_width(rho, g, eta, theta_deg, yield_strength, effusion_rate, wb):
    E= effusion_rate
    wc1 = (24 * E * eta / yield_strength / np.deg2rad(theta_deg)**2) ** (1/3)
    wc2 = ((24 * E * eta)**4 * g * rho / yield_strength**5 / np.deg2rad(theta_deg)**6) ** (1/11)

    
    if wc1 <= 2*wb:
        return wc1
    elif wc2 > 2*wb:
        print(wc1, wc2)
        return wc2
    else:
        print("channel width computation error")
        

def channel_thickness(rho, g, eta, theta_deg, yield_strength, effusion_rate):
    wb = levee_width(rho, g, eta, theta_deg, yield_strength, effusion_rate)
    wc = channel_width(rho, g, eta, theta_deg, yield_strength, effusion_rate, wb)
    wavg = 0.5 *(2* wb + wc)
    h = (yield_strength * wavg / g / rho) ** 0.5
    return h


def cooling_timescale(rho, h, Tstart, Tend, C, flux):
    t_cool = (rho*C*h*(Tstart-Tend)) / flux
    return t_cool

def advancement_timescale(w, h, l, E):
    E= effusion_rate
    volume = l*w*h
    t_adv = volume / E
    return t_adv  

def flow_length():
    pass

In [165]:
def radiative_flux():
    pass

def conductive_flux():
    pass

def flynn_flux(Tsurface):
    flux = 1.07e-13 * Tsurface**4.84 * 1e3
    return flux

In [166]:
## Initial lava conditions ##

## environemnt
g = 8.87
theta_deg = 2

## eruption
effusion_rate = 500          ## m3/s
Tlava = 1400                 ## K
rho = 2600                   ## Basalt density from Davies et al., 2006
phi0_crystal = 0.25
eta0 = 1000                  ## Pas
ys0 = 6500*phi0_crystal**2.85


## Other variables
# L = 100e3                    ## latent heat capacity in J/kg; value from Wittmann et al.2017
# C =  1200                    ## specific heat capacity in J/kg/K; value from Wittman et al. 2017
# k = 1                        ## thermal conductivity in W/m/K from Head and Wilson 1986, and Davies et al., 2005
# kappa = k/rho/c              ## thermal diffusivity in m2/s;

In [167]:
ys0 * 0.12184696791468343

15.235513904729192

In [168]:
Tsurface = Tlava
eta = eta0
ys = ys0
phi_crystal = phi0_crystal


## Emplacement
wb = levee_width(rho, g, eta, theta_deg, ys, effusion_rate)
wc = channel_width(rho, g, eta, theta_deg, ys, effusion_rate, wb)
h = channel_thickness(rho, g, eta, theta_deg, ys, effusion_rate)
hb = levee_thickness(rho, g, eta, theta_deg, ys, effusion_rate)
vel = velocity_Bingham(rho, g, h, eta, theta_deg, ys)

## surface flux
Q = flynn_flux(Tsurface)

56.20069548096858
428.6551502103472 649.3090294709671
56.20069548096858
428.6551502103472 649.3090294709671


In [169]:
Q

180569.1153615039

In [170]:
## calculation of new variables
eta = viscosity(Tlava, Tcurrent, phi_crystal, phi_crystal_max, eta0)
ys = yield_strength()
vel = velocity_Bingham(rho, g, h, eta, theta_deg, yield_strength)

NameError: name 'Tcurrent' is not defined